# Imports

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

# Pré-Processamento do Dataset

Utilizando [função](https://docs.responsibly.ai/_modules/responsibly/dataset/compas.html#COMPASDataset) de pré-processamento do pacote responsibly

In [ ]:
# load of the dataset
#ds = pd.read_csv("../datasets/compas/cox-violent-parsed.csv")

# export the dataset to csv file
#ds.to_csv('dataset.csv', index=False, encoding='utf8')

def preProcess(ds):
    # filter row of:
    # screening arrest between and -30 and 30
    # without recid information (-1)
    # charge degree other than O
    # score text not empty
    ds = ds[(ds['days_b_screening_arrest'] <= 30) & (ds['days_b_screening_arrest'] >= -30) & (ds['is_recid'] != -1) & (ds['c_charge_degree'] != 'O') & (ds['score_text'] != 'N/A')]

    # convert values of jail in and jail out to date/time format
    ds['c_jail_out'] = pd.to_datetime(ds['c_jail_out'])
    ds['c_jail_in'] = pd.to_datetime(ds['c_jail_in'])
    # calculate the prison days
    ds['length_of_stay'] = (ds['c_jail_out'] - ds['c_jail_in'])

    # classificate medium and high score like highscore
    ds['score_factor'] = np.where(ds['score_text'] != 'Low', 'HighScore', 'LowScore')
    # create the prediction values, highscore is 1 and lowscore is 0
    ds['y_pred'] = (ds['score_factor'] == 'HighScore')

    return ds

#ds = preProcess(ds)

# export the dataset pre processed to csv file
#ds.to_csv('dataset-pre-processed.csv', index=False, encoding='utf8')

# Carregamento do Dataset Pré-Processado

In [ ]:
ds = pd.read_csv("dataset-pre-processed.csv")

#drop features not importants to prediction
#ds.drop(ds[['id', 'name', 'first', 'last', 'compas_screening_date', 'dob', 'c_jail_in', 'c_jail_out', 'c_case_number', 'c_offense_date', 'c_arrest_date', 'is_recid', 'r_case_number', 'r_offense_date', 'r_jail_in', 'r_jail_out', 'violent_recid', 'vr_case_number', 'vr_offense_date', 'type_of_assessment', 'decile_score.1', 'score_text', 'screening_date', 'v_type_of_assessment', 'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1', 'y_pred']], axis=1, inplace=True)

# data features to train and test models
data_x = ds[['sex', 'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score', 'juv_misd_count', 'juv_other_count', 'priors_count', 'days_b_screening_arrest', 'c_charge_degree', 'length_of_stay']]

label = ds['is_recid']
prev = ds['y_pred']

# interessante fazer um tratamento das datas de custódia, calcular os dias talvez
# tratar a coluna length_of_stay

In [ ]:
for i in range(len(ds)):
    if ds.loc[i]['is_recid'] != ds.loc[i]['y_pred']:
        print(i, ds.loc[i]['is_recid'], ds.loc[i]['y_pred'])

# for i in range(len(ds)):
#     print(i, ds.loc[i]['event'], ds.loc[i]['y_pred'])

In [ ]:
print(ds['age_cat'].value_counts().sort_values().to_string())

# for i in range(len(ds)):
#     if str(ds.loc[i]['violent_recid']) != 'nan':
#         print(i, ds.loc[i]['violent_recid'])

## Divisão do Dataset para Treino e Teste

In [ ]:
print(len(ds))

qtd70 = (len(ds) * 70) // 100

train = ds.loc[:qtd70]
test = ds.loc[qtd70:]

print(len(train))
print(len(test))

# Carregamento e Execução de Modelos

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

random_forest = RandomForestClassifier()
gradient_boost = GradientBoostingClassifier()

#random_forest.fit(ds)

